# MMA Flow

![alt](../../media/imgs/diag_flow_v2.png)

In [ ]:
import os

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise EnvironmentError("GEMINI_API_KEY not found in environment variables. Please set it before running this notebook.")

In [2]:
from google import genai
import time

client = genai.Client(api_key=GEMINI_API_KEY)

In [ ]:
TEST_VIDEO_FILE = "../../media/splits/box_pollito.mp4"
assert os.path.exists(TEST_VIDEO_FILE), f"Media file {TEST_VIDEO_FILE} does not exist."


In [4]:
def upload_and_wait_for_file_processing(filepath, client, poll_interval=10, max_retries=10):
    uploaded_file = client.files.upload(file=filepath)
    retries = 0
    while uploaded_file.state.name == "PROCESSING":
        print('.', end='', flush=True)
        time.sleep(poll_interval)
        retries += 1
        if retries > max_retries:
            raise TimeoutError("El procesamiento del archivo tomó demasiado tiempo.")
        uploaded_file = client.files.get(name=uploaded_file.name)
    if uploaded_file.state.name == "FAILED":
        raise ValueError(f"Procesamiento fallido: {uploaded_file.state.name}")
    return uploaded_file

myfile = upload_and_wait_for_file_processing(
    filepath=TEST_VIDEO_FILE, # Replace here with slice video file path
    client=client,
    poll_interval=10,
    max_retries=10
)


.

In [30]:
def generate_general_analyst_prompt(deporte="box", lista_skills=None):
    """
    Genera un prompt para el Analista General que actúa como coach virtual,
    proporcionando feedback crítico, constructivo, directo y breve sobre las habilidades del usuario
    en el deporte especificado, registrando eventos relevantes, valorando también las intenciones o aspectos positivos y sugiriendo mejoras.

    También debe ser directo, breve, ve al grano con lo bueno, lo malo y lo que falta mejorar.
    No inventes logros: si hay algo bueno o se nota intención, resáltalo, pero si no hay, dilo tal cual, sin adornar.
    Al final, lista Pros y Contras claros (pueden ser cortos), siempre respaldados con rangos de tiempo concretos/patrones si aplica.
    El feedback debe ayudar a mejorar, sin vueltas ni elogios vacíos.

    Args:
        deporte (str): El deporte a analizar (ej: 'box', 'kickboxing', etc.)
        lista_skills (list of str): Lista de skills a observar (ej: ['técnica', 'defensa', 'movimiento'])
    """
    if not lista_skills:
        lista_skills = ["técnica", "defensa", "movimiento"]

    skills_md = "\n".join([f"- **{skill.capitalize()}**" for skill in lista_skills])
    skills_inline = ", ".join([skill.capitalize() for skill in lista_skills])

    prompt = f"""
**ROL:** Analista General, coach virtual de **{deporte.upper()}**.
Observa el video y dame feedback directo, claro y breve: dime lo que va bien, lo que va mal y lo que se puede mejorar. Sin adornos, sin elogios vacíos.

**ENFOQUE PRINCIPAL**
Observa especialmente estas habilidades:
{skills_md}

**CÓMO HACERLO:**
- Registra, segundo a segundo, eventos relevantes para estas skills: {skills_inline}
- Si no pasa nada relevante en algún segundo, dilo igual (por ejemplo, sin acción).
- Identifica logros o buenas intenciones (ej: atacar primero, moverse bien, creatividad) y errores claros. Si hay actitud o intención buena aunque falle la ejecución, menciónalo brevemente.
- No adornes las críticas. Sé claro y conciso: ¿qué está bien? ¿qué está mal? ¿qué se repite?
- Da un feedback accionable, concreto y corto en cada comentario.
- Si detectas patrones de errores, marca que encontraste un patrón y el rango de tiempo.
- Para cada observación importante, da rango de timestamps (ej: "00:15-00:18").
- Usa una tabla Markdown de **CUATRO columnas**: **Tiempo (MM:SS)**, **Acción Observada**, **Skill Asociada** y **Feedback/Crítica**.

**EJEMPLO DE TABLA:**
| Tiempo (MM:SS) | Acción Observada | Skill Asociada | Feedback/Crítica |
| :---: | :--- | :---: | :--- |
| 00:00 | Guardia alta, pies juntos | Defensa | Bien arriba las manos, pero corrige el balance |
| 00:01 | Jab al avanzar, baja la derecha | Técnica | Jab correcto, no bajes la otra mano |
| 00:02 | Amaga pero predecible | Movimiento | Buena intención, pero más variedad en la finta |
| 00:03 | Nada relevante | Movimiento | Falta de iniciativa este segundo |

**AL FINAL:**
1. Dime en pocas líneas las debilidades principales por skill y 2-3 cosas claras para mejorar.
2. Si hay patrones de error, explica el patrón brevemente y qué tramo de tiempo lo repite.
3. Haz una lista breve de **Pros** (si hay cosas buenas, di cuáles y el rango de tiempo; si no hay pros, dilo directo).
4. Haz una lista breve de **Contras** (errores graves o recurrentes y rango de tiempo; si no hay, dilo directo).
5. Señala si alguna skill destaca mucho (por buena o mala) para enfocar el entrenamiento.

**TONO:** Honesto, directo, crítico y constructivo. Nada de adornos: sólo lo necesario para que se entienda qué está bien, qué está mal, y cómo avanzar como deportista en {skills_inline}.

**NOTA:** Tu misión es que el practicante mejore: ve al grano, no omitas errores ni inventes logros, y resalta solamente lo realmente positivo o la intención clara si la hay. Pros/Contras siempre con timestamps.
"""
    return prompt

In [ ]:
# Generar el prompt del analista general
prompt_general = generate_general_analyst_prompt(lista_skills=["agresividad", "defensa", "movimiento"])

# Ejecutar el análisis general (ground truth)
response_general = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents=[myfile, prompt_general], 
    config={
        "temperature": 0.0,
        "topP": 0.95,
        "seed": 42,
    }
)

# Guardar el análisis general
general_analysis = response_general.text
print(general_analysis)

¡Entendido! Aquí tienes el feedback directo y conciso sobre el video, enfocado en las habilidades de Agresividad, Defensa y Movimiento.

| Tiempo (MM:SS) | Acción Observada | Skill Asociada | Feedback/Crítica |
| :---: | :--- | :---: | :--- |
| 00:00-00:09 | Flexiona bíceps | Agresividad | No hay agresividad real, solo poses. Falta intención de ataque. |
| 00:00-00:09 | Movimientos rígidos | Movimiento | Movimientos muy robóticos y sin fluidez. Necesitas soltarte más. |
| 00:00-00:09 | Brazos muy abiertos | Defensa | La guardia es inexistente. Brazos muy separados del cuerpo, vulnerable a ataques al centro. |

**Debilidades Principales:**

*   **Agresividad:** Nula. Falta total de intención de ataque o demostración de poder ofensivo.
*   **Defensa:** Inexistente. Guardia baja y brazos separados del cuerpo.
*   **Movimiento:** Rígido y sin propósito. Falta fluidez y coordinación.

**Cosas Claras para Mejorar:**

1.  **Guardia:** Mantén los codos pegados al cuerpo y las manos cerca de la

In [ ]:

# myfile2 = upload_and_wait_for_file_processing(
#     filepath="../../media/splits/box_violento.mp4", # Replace here with slice video file path
#     client=client,
#     poll_interval=10,
#     max_retries=10
# )

# Ejecutar el análisis general (ground truth)
response_general2 = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents=[myfile2, prompt_general], 
    config={
        "temperature": 0.0,
        "topP": 0.95,
        "seed": 42,
    }
)

# Guardar el análisis general
general_analysis2 = response_general2.text
print(general_analysis2)

¡Entendido! Aquí tienes el feedback directo y conciso sobre el video, enfocado en las habilidades de Agresividad, Defensa y Movimiento.

| Tiempo (MM:SS) | Acción Observada | Skill Asociada | Feedback/Crítica |
| :---: | :--- | :---: | :--- |
| 00:00-00:06 | Movimientos circulares de brazos | Agresividad | Falta de dirección y propósito en los golpes. No hay intención de conectar. |
| 00:00-00:06 | Movimientos amplios y descoordinados | Movimiento | Movimientos ineficientes y sin balance. Reduce la amplitud y busca equilibrio. |
| 00:00-00:06 | Exposición total | Defensa | Ausencia total de guardia. Peligroso. |
| 00:03-00:06 | Sonrisa a cámara | Agresividad | Concentración en la tarea. No es momento de sonreír. |

**Debilidades Principales:**

*   **Agresividad:** Falta de intención real de ataque, movimientos sin dirección.
*   **Defensa:** Ausencia total de guardia, exposición constante.
*   **Movimiento:** Descoordinación, falta de balance, movimientos ineficientes.

**Cosas Claras

In [ ]:

# myfile3 = upload_and_wait_for_file_processing(
#     filepath="../../media/splits/box_medio_tecnico.mp4", # Replace here with slice video file path
#     client=client,
#     poll_interval=10,
#     max_retries=10
# )

# Ejecutar el análisis general (ground truth)
response_general3 = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents=[myfile3, prompt_general], 
    config={
        "temperature": 0.0,
        "topP": 0.95,
        "seed": 42,
    }
)

# Guardar el análisis general
general_analysis3 = response_general3.text
print(general_analysis3)

¡Entendido! Aquí tienes el análisis directo y conciso del video, enfocado en las habilidades solicitadas:

| Tiempo (MM:SS) | Acción Observada | Skill Asociada | Feedback/Crítica |
| :---: | :--- | :---: | :--- |
| 00:00-00:02 | Movimientos rápidos de manos | Agresividad | Falta de potencia y dirección en los golpes. |
| 00:00-00:02 | Guardia baja | Defensa | Mantén la guardia alta para proteger el rostro. |
| 00:00-00:02 | Pies estáticos | Movimiento | Necesitas juego de pies para mejorar el posicionamiento y la movilidad. |
| 00:03-00:05 | Intento de jab | Agresividad | El jab es débil y sin seguimiento. |
| 00:03-00:05 | Guardia baja | Defensa | La guardia sigue baja, exponiendo el rostro. |
| 00:03-00:05 | Movimiento limitado | Movimiento | Falta de desplazamiento lateral y circular. |
| 00:06-00:08 | Combinación de golpes | Agresividad | La combinación es lenta y telegrafiada. |
| 00:06-00:08 | Guardia baja | Defensa | La guardia sigue siendo un problema constante. |
| 00:06-00:08

In [ ]:
# myfile4 = upload_and_wait_for_file_processing(
#     filepath="../../media/splits/box_pro_i_guess.mp4", # Replace here with slice video file path
#     client=client,
#     poll_interval=10,
#     max_retries=10
# )

# Ejecutar el análisis general (ground truth)
response_general4 = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents=[myfile4, prompt_general], 
    config={
        "temperature": 0.0,
        "topP": 0.95,
        "seed": 42,
    }
)

# Guardar el análisis general
general_analysis4 = response_general4.text
print(general_analysis4)

¡Entendido! Aquí tienes el análisis del video, enfocado en las habilidades solicitadas y con el tono directo y constructivo que pediste.

| Tiempo (MM:SS) | Acción Observada | Skill Asociada | Feedback/Crítica |
| :---: | :--- | :---: | :--- |
| 00:06 | Lanza un jab al aire | Agresividad | El jab es lento y telegrafiado. Necesitas más velocidad y ocultar la intención. |
| 00:07 | Se ajusta la ropa | Concentración | Evita distracciones. Mantén el foco en el entrenamiento. |
| 00:10 | Jab al aire | Agresividad | Jab lento y sin potencia. Trabaja la explosividad en el golpe. |
| 00:11 | Lanza un gancho al aire | Agresividad | El gancho es muy abierto y predecible. Cierra más el ángulo y usa el cuerpo para generar potencia. |
| 00:12 | Se mueve lateralmente | Movimiento | Movimiento lateral básico, pero sin propósito claro. Define mejor la dirección y el objetivo del movimiento. |
| 00:13 | Jab al aire | Agresividad | Jab sin potencia ni velocidad. Necesitas trabajar la técnica y la fuerza

In [ ]:
# myfile5 = upload_and_wait_for_file_processing(
#     filepath="../../media/splits/box_pro.mp4", # Replace here with slice video file path
#     client=client,
#     poll_interval=10,
#     max_retries=10
# )

prompt_general5 = generate_general_analyst_prompt(lista_skills=["velocidad de golpes", "soltura de hombros", "fluidez"])

# Ejecutar el análisis general (ground truth)
response_general5 = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents=[myfile5, prompt_general5], 
    config={
        "temperature": 0.0,
        "topP": 0.95,
        "seed": 42,
    }
)

# Guardar el análisis general
general_analysis5 = response_general5.text
print(general_analysis5)

Aquí tienes el feedback directo y sin adornos:

| Tiempo (MM:SS) | Acción Observada | Skill Asociada | Feedback/Crítica |
| :---: | :--- | :---: | :--- |
| 00:00-00:03 | Posición estática, sin movimiento de pies. | Fluidez | Falta de movimiento preparatorio, el cuerpo está rígido. |
| 00:03-00:05 | Inicio de golpes muy cortos y rápidos. | Velocidad de golpes | Los golpes son solo de brazo, sin extensión ni rotación de cadera. No es velocidad de golpe, es velocidad de "twitch". |
| 00:05-00:07 | Mantiene golpes cortos, hombros tensos. | Soltura de hombros | Hombros visiblemente contraídos, no hay relajación ni rotación para generar potencia. |
| 00:07-00:09 | Aumenta la frecuencia de los golpes, pero siguen siendo cortos. | Fluidez | El movimiento es segmentado (solo brazos), no hay fluidez corporal. El tronco y las piernas están estáticos. |
| 00:09-00:10 | Finaliza con el mismo patrón de golpes. | Velocidad de golpes | La velocidad es de repetición de un movimiento incompleto, no de u